In [1]:
import pandas as pd
import numpy as np

In [2]:
def getPricesFromCSV(file_path, reference_dates):
    # get all the data from the csv file where the dates are in the reference_dates list
    df = pd.read_csv(reference_dates, skiprows=15)
    df["Date"] = pd.to_datetime(df["Date"])
    date_series = df["Date"]

    df = pd.read_csv(file_path)
    df["Date"] = pd.to_datetime(df["Date"], format="%Y%m%d")
    df_filtered = df[df["Date"].isin(date_series)]
    return df_filtered

In [3]:
dataFrame_H5A4 = getPricesFromCSV("/Volumes/Raghav_DATA/Capitalfund/fixed-income/Data Files/H5A4.csv", "/Volumes/Raghav_DATA/Capitalfund/fixed-income/Data Files/MLH5A4_Price_History(Fixed Income).xls")
dataFrame_SPY = getPricesFromCSV("/Volumes/Raghav_DATA/Capitalfund/fixed-income/Data Files/SPY 1.csv", "/Volumes/Raghav_DATA/Capitalfund/fixed-income/Data Files/MLH5A4_Price_History(Fixed Income).xls")

In [4]:
def create_bmom(dataframe, weights, price_col="Price"):
    data = dataframe.dropna().sort_values("Date").set_index("Date")

    px = data[price_col]
    momentums = {}
    weighted_momentums = {}

    for lookback, weight in weights.items():
        momentums[lookback] = px.pct_change(lookback)
        weighted_momentums[lookback] = momentums[lookback] * weight

    wm_df = pd.concat(weighted_momentums, axis=1)

    # Find first index where ALL lookbacks are valid
    first_valid = wm_df.dropna().index[0]

    # Sum only from that index onward
    bmom = wm_df.loc[first_valid:].sum(axis=1)

    daily_returns = px.pct_change(1)
    dates = bmom.index

    return bmom, momentums, dates, daily_returns

bmom_H5A4, momentums_H5A4, dates_H5A4, daily_returns_H5A4 = create_bmom(
    dataFrame_H5A4,
    {20: 0.15, 60: 0.35, 120: 0.35, 240: 0.15},
    "H5A4"
)

bmom_SPY, momentums_SPY, dates_SPY, daily_returns_SPY = create_bmom(
    dataFrame_SPY,
    {20: 0.15, 60: 0.35, 120: 0.35, 240: 0.15},
    "SPY_adj_close"
)





In [5]:
def strat1_signals(bmom_asset, bmom_spy, mom240_asset, thresh=-0.03, init_wgt=0):
    # Use asset index as the master timeline (like the sheet's Date column)
    idx = bmom_asset.index
    a = bmom_asset.astype(float)
    s = bmom_spy.reindex(idx).astype(float)
    m240 = mom240_asset.reindex(idx).astype(float)

    # Cross-down conditions (NaNs will naturally evaluate to False in comparisons)
    cond1 = (s.shift(1) > thresh) & (s < thresh) & (a < thresh)
    cond2 = (a.shift(1) > thresh) & (a < thresh) & (s < thresh)

    risk_off = (cond1 | cond2) & (m240 < 0)

    # Desired signal: 0 on risk_off, 1 if a>0, else NaN (to be forward-filled)
    sig = pd.Series(np.nan, index=idx, dtype="float64")
    sig = sig.mask(risk_off, 0)
    sig = sig.mask(~risk_off & (a > 0), 1)

    # Spreadsheet-style initialization + carry forward
    sig.iloc[0] = init_wgt
    sig = sig.ffill().astype(int)

    return sig


In [6]:
signals_strat1 = strat1_signals(bmom_H5A4, bmom_SPY, momentums_H5A4[240], thresh=-0.03)

In [7]:
def strat2_from_strat1(strat1: pd.Series,
                       hyg_ret: pd.Series,
                       drawdiffthresh: float = 0.10) -> pd.Series:
    idx = strat1.index
    r_hyg = hyg_ret.reindex(idx).fillna(0.0)

    # Strat1 returns use previous-day weight (typical backtest convention)
    r_strat1 = r_hyg * strat1.shift(1).fillna(0)

    def drawdown(r: pd.Series) -> pd.Series:
        eq = (1.0 + r).cumprod()
        return eq / eq.cummax() - 1

    dd_strat1 = drawdown(r_strat1)
    dd_hyg = drawdown(r_hyg)


    dd_diff = dd_strat1 - dd_hyg


    override_trigger = dd_diff > drawdiffthresh
    
    # Identify start of each override regime
    override_start = override_trigger & (~override_trigger.shift(1, fill_value=False))
    
    # Each override regime gets a unique ID
    regime_id = override_start.cumsum()
    
    # Determine where an override regime should end:
    # first date AFTER regime start where strat1 == 1
    end_override = (regime_id > 0) & (strat1 == 1)
    
    # For each regime, find first exit index
    first_exit = (
        end_override.groupby(regime_id)
        .transform(lambda x: x & (~x.shift(1, fill_value=False)))
    )
    
    # Build active override mask
    active_override = (
        (regime_id > 0)
        & ~(first_exit.groupby(regime_id).cumsum().astype(bool))
    )
    
    # Strat2 logic
    strat2 = strat1.copy()
    strat2[active_override] = 1
    strat2[override_trigger] = 1   # threshold always wins
    
    strat2.iloc[0] = 0
    
    return strat2



strat2_signals = (strat2_from_strat1(signals_strat1, daily_returns_H5A4, drawdiffthresh=0.10))
strat2_zero = strat2_signals[strat2_signals == 0]


In [8]:
def analyze_strat(signals: pd.Series, daily_returns: pd.Series, file_path):
    idx = signals.index
    r_strat = daily_returns.reindex(idx).fillna(0.0) * signals.shift(1).fillna(0)
    
    def drawdown(r: pd.Series) -> pd.Series:
        eq = (1.0 + r).cumprod()
        return eq / eq.cummax() - 1

    def equity_curve(r: pd.Series) -> pd.Series:
        return (1.0 + r).cumprod()
    
    def annualized_return(r: pd.Series) -> float:
        equity = equity_curve(r)
        return equity.iloc[-1] ** (252 / len(r)) - 1  # Annualized return
    def volatility(r: pd.Series) -> float:
        return r.std() * np.sqrt(252)  # Annualized volatility
    def sharpe_ratio(r: pd.Series, risk_free_rate=0.0) -> float:
        ann_return = annualized_return(r)
        ann_vol = volatility(r)
        return ann_return / ann_vol if ann_vol > 0 else np.nan
    print("file:", file_path)
    print(f"Annualized Return: {annualized_return(r_strat):.2}")
    print(f"Annualized Volatility: {volatility(r_strat):.2}")
    print(f"Sharpe Ratio: {sharpe_ratio(r_strat):.2}")
    
    #Write to CSV
    output_df = pd.DataFrame({
        "Date": idx,
        "Signal": signals,
        "Daily Return": r_strat,
        "Drawdown": drawdown(r_strat),
        "Equity Curve": equity_curve(r_strat)
    })
    output_df.to_csv(file_path, index=False)


analyze_strat(strat2_signals, daily_returns_H5A4, "/Volumes/Raghav_DATA/Capitalfund/fixed-income/strat2_output.csv")
analyze_strat(signals_strat1, daily_returns_H5A4, "/Volumes/Raghav_DATA/Capitalfund/fixed-income/strat1_output.csv")


file: /Volumes/Raghav_DATA/Capitalfund/fixed-income/strat2_output.csv
Annualized Return: 0.012
Annualized Volatility: 0.044
Sharpe Ratio: 0.28
file: /Volumes/Raghav_DATA/Capitalfund/fixed-income/strat1_output.csv
Annualized Return: 0.0087
Annualized Volatility: 0.036
Sharpe Ratio: 0.24
